In [1]:
# !pip install spacy
# !pip install PyMuPDF
# !python -m spacy download en_core_web_lg

In [2]:
import sys,fitz
def convert(fname):
    doc = fitz.open(fname)
    text = ""
    for page in doc:
        text = text +str(page.get_text())
    tx = " ".join(text.split("\n"))
    return tx

In [3]:
import re
import spacy
nlp = spacy.load("en_core_web_lg")
skills = "jz_skill_patterns.jsonl"
ruler = nlp.add_pipe("entity_ruler",before = "ner")
ruler.from_disk(skills)
pattern = [{
    "label":"EMAIL","pattern":[{"TEXT":{"REGEX" : "([^@|\s]+@[^@]+\.[^@|\s]+)"}}
                              ]},
    {
        "label":"MOBILE","pattern":[{"TEXT":{"REGEX" : "(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?"}}
                                   ]
    }
]

In [4]:
ruler.add_patterns(pattern)

In [5]:
text = convert("Mohamed_fathy.pdf")

In [6]:
doc = nlp(text)

In [7]:
# from spacy import displacy
# displacy.render(doc, style="ent", jupyter=True)

In [8]:
dic = {}
skills = []
i = 0
for ent in doc.ents:
    if ent.label_ == "PERSON" and i == 0:
        dic["PERSON"] = ent.text
        i += 1
    if ent.label_ == "EMAIL":
        dic["EMAIL"] = ent.text
    if ent.label_ == "MOBILE":
        dic["MOBILE"] = ent.text
    if ent.label_ == "SKILL":
        skills.append(ent.text)
skills = [i.capitalize() for i in set([i.lower() for i in skills])]
dic["SKILLS"] = skills
dic["location"] = [entity.text for entity in doc.ents if entity.label_ == 'GPE'][0]

In [9]:
portfolio=[]
from spacy.lang.tokenizer_exceptions import URL_PATTERN
custom_infixes = [URL_PATTERN[1:-1]] + list(nlp.Defaults.infixes)
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(custom_infixes).finditer
for t in nlp.tokenizer.explain(doc.text):
    if t[0] == "URL_MATCH":
        if "/" in t[1]:
            portfolio.append(t[1])

In [10]:
import re
rx = re.compile(r"(\d+(?:-\d+)?\+?)\s*(years?)", re.I)
exp_temp = rx.search(text)
if exp_temp:
    dic["experience"] = " ".join(exp_temp.groups())

In [11]:
dic["Link"] = portfolio

In [12]:
dic

{'PERSON': 'MOHAMED AHMED FATHY',
 'MOBILE': '+201009978304',
 'EMAIL': 'Mohamed.ahmedmoreb@gmail.com',
 'SKILLS': ['Database',
  'Redux',
  'Angular',
  'Graphql',
  'Html',
  'Django',
  'Mysql',
  'Dart',
  'Data structure',
  'Wordpress',
  'Google drive',
  'Azure',
  'Typescript',
  'Engineering',
  'React',
  'C++',
  'Front end',
  'Heroku',
  'Software',
  'Mongodb',
  'Javascript',
  'Android',
  'Google',
  'C',
  'Algorithm',
  'Trello',
  'Docker',
  'Docker compose',
  'Search engine',
  'Php',
  'Sql',
  'Laravel',
  'Flutter',
  'Testing',
  'Workflow',
  'Flask',
  'Finance',
  'Python',
  'Mean',
  'Java',
  'Css',
  'Design'],
 'location': 'Cairo',
 'experience': '1+ year',
 'Link': ['https://github.com/mhmdahmedfathi/Search-Engine',
  'https://github.com/mhmdahmedfathi/Tasks_NTI',
  'https://github.com/mhmdahmedfathi/NTI_Project']}